In [17]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize

In [18]:
url_df=pd.read_excel('Test.xlsx')
url_df.shape

(5, 2)

In [19]:
url_df.head()

,URL_ID,URL
0,1,https://en.wikipedia.org/wiki/Hygrocybe_miniata
1,2,https://en.wikipedia.org/wiki/Hygrocybe_appala...
2,3,https://en.wikipedia.org/wiki/Fungus
3,4,https://en.wikipedia.org/wiki/Chromista
4,5,https://en.wikipedia.org/wiki/Diatom


Creating an extracted DataFrame with webscrapped data

In [20]:
xtrctd_df=pd.DataFrame()
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"
    }
for i in range(5):
    result = requests.get(url_df['URL'][i],headers=headers)
    soup = BeautifulSoup(result.text,"lxml")
    xtrctd_df.loc[i,'Url']=url_df['URL'][i]
    xtrctd_df.loc[i,'Title']=soup.select('title')[0].getText()
    a=str()
    for j in soup.select('p'):
        a=a+j.getText()
    xtrctd_df.loc[i,'Text']=a
    xtrctd_df['Text'][i]=xtrctd_df['Text'][i].replace('\n',' ')

In [21]:
xtrctd_df

,Url,Title,Text
0,https://en.wikipedia.org/wiki/Hygrocybe_miniata,Hygrocybe miniata - Wikipedia,Hygrocybe strangulata (P.D.Orton) Svrcek Hygr...
1,https://en.wikipedia.org/wiki/Hygrocybe_appala...,Hygrocybe appalachianensis - Wikipedia,"Hygrocybe appalachianensis, commonly known as..."
2,https://en.wikipedia.org/wiki/Fungus,Fungus - Wikipedia,A fungus (plural: fungi[2] or funguses[3]) is...
3,https://en.wikipedia.org/wiki/Chromista,Chromista - Wikipedia,Chromista is a biological kingdom consisting ...
4,https://en.wikipedia.org/wiki/Diatom,Diatom - Wikipedia,Diatom (Neo-Latin diatoma)[a] refers to any m...


Text Processing

In [22]:
from nltk.corpus import stopwords
import string
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [23]:
def sentiment_analyse(sentiment_text):
    score=SentimentIntensityAnalyzer().polarity_scores(sentiment_text)
    # print(score)
    neg=score["neg"]
    pos=score["pos"]
    return pos,neg

In [24]:
def Check_Vow(string):
    vowels = "AaEeIiOoUu"
    final = [each for each in string if each in vowels]
    return (len(final))

In [25]:
def avgwlen(words):
    leng=[]
    for word in words:
        leng.append(len(word))        
    return np.mean(leng)


In [26]:
xtrctd_df.loc[0,'Url']

'https://en.wikipedia.org/wiki/Hygrocybe_miniata'

In [29]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
new_df=pd.DataFrame()
cleanmessyy=xtrctd_df['Text'].apply(text_process)
for i in range(5):
    new_df.loc[i,'Url']=xtrctd_df.loc[i,'Url']
    cleanmess=cleanmessyy[i]
    tokenized_word_data =word_tokenize(xtrctd_df['Text'][i])
    tokenized_sentance_data=sent_tokenize(xtrctd_df['Text'][i])
    new_df.loc[i,['Positivity_score','Negativity_score']]=sentiment_analyse(xtrctd_df['Text'][i])
    avgsentlen=np.mean([len(i) for i in tokenized_sentance_data])
    new_df.loc[i,'Avg_sentence_length']=avgsentlen
    avgwrdspersent=len(tokenized_word_data)/len(tokenized_sentance_data)
    new_df.loc[i,'Avg_words_per_sentence']=avgwrdspersent
    wordcnt=len(tokenized_word_data)
    new_df.loc[i,'Word_Count']=wordcnt
    new_df.loc[i,'Avg_word_length']=avgwlen(tokenized_word_data)

In [30]:
new_df

,Url,Positivity_score,Negativity_score,Avg_sentence_length,Avg_words_per_sentence,Word_Count,Avg_word_length
0,https://en.wikipedia.org/wiki/Hygrocybe_miniata,0.042,0.016,103.307692,21.538462,560.0,4.046429
1,https://en.wikipedia.org/wiki/Hygrocybe_appala...,0.025,0.012,104.895522,20.522388,1375.0,4.341818
2,https://en.wikipedia.org/wiki/Fungus,0.057,0.035,164.662304,29.193717,11152.0,4.861101
3,https://en.wikipedia.org/wiki/Chromista,0.029,0.016,150.857143,26.119048,1097.0,5.018232
4,https://en.wikipedia.org/wiki/Diatom,0.062,0.027,159.332143,29.035714,8130.0,4.684625
